<a href="https://colab.research.google.com/github/NArtEvents11/hello-world/blob/master/ML_Pytanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

In [0]:
df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [25]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [26]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


In [27]:
print(df.describe())

       PassengerId    Survived      Pclass  ...       SibSp       Parch        Fare
count   891.000000  891.000000  891.000000  ...  891.000000  891.000000  891.000000
mean    446.000000    0.383838    2.308642  ...    0.523008    0.381594   32.204208
std     257.353842    0.486592    0.836071  ...    1.102743    0.806057   49.693429
min       1.000000    0.000000    1.000000  ...    0.000000    0.000000    0.000000
25%     223.500000    0.000000    2.000000  ...    0.000000    0.000000    7.910400
50%     446.000000    0.000000    3.000000  ...    0.000000    0.000000   14.454200
75%     668.500000    1.000000    3.000000  ...    1.000000    0.000000   31.000000
max     891.000000    1.000000    3.000000  ...    8.000000    6.000000  512.329200

[8 rows x 7 columns]


In [28]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [0]:
def get_title(name):
  if '.' in name:
    return name.split(',')[1].split('.')[0].strip()
  else:
    return 'Unknown'

In [30]:

# A list with the all different titles 
titles = sorted(set([x for x in df.Name.map(lambda x: get_title(x))]))
print('Different titles found on the dataset: ')
print(len(titles), ': ', titles)


Different titles found on the dataset: 
17 :  ['Capt', 'Col', 'Don', 'Dr', 'Jonkheer', 'Lady', 'Major', 'Master', 'Miss', 'Mlle', 'Mme', 'Mr', 'Mrs', 'Ms', 'Rev', 'Sir', 'the Countess']


In [0]:
#Normalize the tiles
def replace_titles(x):
  title = x['Title']
  if title in ['Capt','Col', 'Major']:
    return 'Officer'
  elif title in ['Jonkheer','Don','the Countess', 'Dona', 'Lady','Sir']:
    return 'Royalty'
  elif title in ['the Countess','Mme','Lady']:
    return 'Mrs'
  elif title in ['Mlle','Ms']:
    return 'Miss'
  else:
    return title


In [0]:
df['Title']= df['Name'].map(lambda x: get_title(x))

In [0]:
df['Title']= df.apply(replace_titles, axis=1)

In [34]:
print(df.Title.value_counts())

Mr         517
Miss       185
Mrs        126
Master      40
Dr           7
Rev          6
Officer      5
Royalty      5
Name: Title, dtype: int64


In [0]:
df['Age'].fillna(df['Age'].median(), inplace = True)
df['Fare'].fillna(df['Fare'].median(), inplace = True)
df['Embarked'].fillna('S', inplace = True)
df.drop('Cabin', axis = 1, inplace = True)
df.drop('Ticket', axis = 1, inplace = True)
df.drop('Name', axis = 1, inplace = True)
df.Sex.replace(('male','female'),(0,1), inplace = True)
df.Embarked.replace(("S",'C','Q'),(0,1,2), inplace= True)
df.Title.replace(('Mr','Miss','Mrs','Master','Dr','Rev','Officer','Royalty'),(0,1,2,3,4,5,6,7), inplace = True)

In [36]:
print(df.isnull().sum())
print(df['Sex'].sample(5))
print(df['Embarked'].sample(5))
print(df.columns)

PassengerId    0
Survived       0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
Title          0
dtype: int64
79     1
148    0
142    1
400    0
126    0
Name: Sex, dtype: int64
612    2
601    0
127    0
123    0
0      0
Name: Embarked, dtype: int64
Index(['PassengerId', 'Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch',
       'Fare', 'Embarked', 'Title'],
      dtype='object')


In [37]:
corr = df.corr()
corr.Survived.sort_values(ascending=False)

Survived       1.000000
Sex            0.543351
Title          0.361340
Fare           0.257307
Embarked       0.106811
Parch          0.081629
PassengerId   -0.005007
SibSp         -0.035322
Age           -0.064910
Pclass        -0.338481
Name: Survived, dtype: float64

In [0]:
from sklearn.model_selection import train_test_split

x = df.drop(['Survived','PassengerId'], axis = 1)
y = df['Survived']
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = 0.1)

In [39]:
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

randomforest = RandomForestClassifier()
randomforest.fit(x_train, y_train)
y_pred = randomforest.predict(x_val)
acc_randomforest = round(accuracy_score(y_pred, y_val) * 100, 2)
print('Accuracy: {}'.format(acc_randomforest))

pickle.dump(randomforest, open('titanic_model.sav', 'wb'))

Accuracy: 81.11


In [0]:
df_test = pd.read_csv('test.csv')

In [0]:
df_test['Title']= df_test['Name'].map(lambda x: get_title(x))
df_test['Title']= df_test.apply(replace_titles, axis=1)
ids = df_test['PassengerId']

df_test['Age'].fillna(df_test['Age'].median(), inplace = True)
df_test['Fare'].fillna(df_test['Fare'].median(), inplace = True)
df_test['Embarked'].fillna('S', inplace = True)
df_test.drop('Cabin', axis = 1, inplace = True)
df_test.drop('Ticket', axis = 1, inplace = True)
df_test.drop('Name', axis = 1, inplace = True)
df_test.drop('PassengerId', axis=1, inplace=True)
df_test.Sex.replace(('male','female'),(0,1), inplace = True)
df_test.Embarked.replace(("S",'C','Q'),(0,1,2), inplace= True)
df_test.Title.replace(('Mr','Miss','Mrs','Master','Dr','Rev','Officer','Royalty'),(0,1,2,3,4,5,6,7), inplace = True)

In [0]:
predictions = randomforest.predict(df_test)
output = pd.DataFrame({ 'PassengerId' : ids, 'Survived': predictions })
output.to_csv('submission.csv', index=False)

In [0]:
from google.colab import files
files.download('submission.csv')